# Free surface model generator


## Theory

Atomic simulations of free surfaces are typically evaluated in the following way:

1. A bulk atomic system is constructed such that the crystal plane of interest coincides with one of the system's boundaries.

2. The cohesive energy of the bulk system is evaluated with all boundary conditions periodic.

3. The free surface is inserted by either making one of the boundaries non-periodic, or greatly expanding said boundary dimension. 

Coordinate system vectors:

- $a_{unit}, b_{unit}, c_{unit}$ refer to the lattice vectors of a standard crystal unit cell.

- $a_{box}, b_{box}, c_{box}$ refer to the box vectors of the full atomic system.

- $[u_a, v_a, w_a], [u_b, v_b, w_b], [u_c, v_c, w_c]$ are the crystallographic Miller \[uvw\] indices coinciding with the three system box vectors. 

The unit and box vectors can be related using the \[uvw\] indices as:
        
$$ a_{box} = u_a a_{unit} + v_a b_{unit} + w_a c_{unit}$$
$$ b_{box} = u_b a_{unit} + v_b b_{unit} + w_b c_{unit}$$
$$ c_{box} = u_c a_{unit} + v_c b_{unit} + w_c c_{unit}$$

To ensure the generated system is periodic for calculation step \#2, the box vectors must be full periodic lattice steps. In other words, all the hkl's must be integers. 

Generating the atomic system (step \#1) requires that two of the system's box vectors be contained within the slip plane of interest. Note that this is *not* the same as taking one of the box vectors to be perpendicular to the slip plane. [Sun and Cedar]() showed that this could be done by identifying three points within the same slip plane, and using the distance vectors between pairs of those points to obtain the two box vectors. The Miller (hkl) plane indices indicate the intercepts of the crystallographic plane relative to the unit cell, which can be used to obtain the three points:

- An (hkl) plane has intercepts at $\frac{a_{unit}}{h}$, $\frac{b_{unit}}{k}$ and $\frac{c_{unit}}{l}$. 

- An (hk0) plane has the intercepts $\frac{a_{unit}}{h}$ and $\frac{b_{unit}}{k}$, and a third point can be obtained by adding $c_{unit}$ to either intercept.  The (h0l) and (0kl) planes can be similarly obtained.

- An (h00) plane has an intercept $\frac{a_{unit}}{h}$, and two other points can be obtained by adding either $b_{unit}$ or $c_{unit}$ to the intercept.  The (0k0) and (00l) planes can be similarly obtained.

As the two resulting in-plane vectors are in terms of the unit cell vectors, they can be written solely in terms of \[uvw\] indices (and therefore system-independent).  For simplicity, we'll take $a_{box}$ and $b_{box}$ to be the in-plane vectors.

- (hkl):
    
    - $[u_a, v_a, w_a] = [\frac{-M}{h}, \frac{M}{k}, 0]$
    - $[u_b, v_b, w_b] = [\frac{-M}{h}, 0, \frac{M}{l}]$

- (hk0):

    - $[u_a, v_a, w_a] = [\frac{-M}{h}, \frac{M}{k}, 0]$
    - $[u_b, v_b, w_b] = [0, 0, 1]$
    
- (h00):

    - $[u_a, v_a, w_a] = [0, 1, 0]$
    - $[u_b, v_b, w_b] = [0, 0, 1]$
    
where $M$ is a multiplier to ensure the \[uvw\]'s are integers.

The $c_{box}$ can be any vector not in the slip plane (which it can't be for the system to have volume!). Practically, it should be defined such that $[u_c, v_c, w_c]$ are small integers and that $c_{box}$ is close to the slip plane normal.

**Note**: In LAMMPS, the resulting system is transformed to adhere to the box vectors limitations:

- $a'_{box} = [lx, 0, 0]$

- $b'_{box} = [xy, ly, 0]$

- $c'_{box} = [xz, yz, lz]$

This transformation results in the free surface plane being normal to the Cartesian z-axis as only the $c'_{box}$ vector has a z-component.

## Setup

**Library imports**

In [1]:
import uuid

import numpy as np

from DataModelDict import DataModelDict as DM

import atomman as am

Define **surface_basis()** that generates uvw rotation sets for a given hkl.

In [2]:
def surface_basis(hkl, box=am.Box(), n=None):
    """
    Uses free surface in-plane vector determination algorithm by W. Sun and G. Cedar,
    Surface Science, 617, 53-59 (2013).
    
    Parameters
    ----------
    hkl : array-like object
        Integer (hkl) indices for the surface plane being created.
    box : atomman.Box, optional
        The box object associated with the unit cell. Used to identify the best uvw set for
        the c direction.  Default value uses a cubic box.
    n : int
        Max uvw index value to use in identifying the best uvw set for the c direction.
        If not given, will use the largest uvw index found for the other two uvw sets.
    """
    hkl = np.asarray(hkl, dtype=int)
    
    if hkl[0] != 0:
        if hkl[1] != 0:
            if hkl[2] != 0:
                # hkl solution
                m = hkl[0] * hkl[1] * hkl[2]
                a_uvw = np.array([-m/hkl[0], m/hkl[1], 0], dtype=int)
                b_uvw = np.array([-m/hkl[0], 0, m/hkl[2]], dtype=int)
            else:
                # hk0 solution
                m = hkl[0] * hkl[1]
                a_uvw = np.array([-m/hkl[0], m/hkl[1], 0], dtype=int)
                b_uvw = np.array([0, 0, 1], dtype=int)
        else:
            if hkl[2] != 0:
                # h0l solution
                m = hkl[0] * hkl[2]
                a_uvw = np.array([m/hkl[0], 0, -m/hkl[2]], dtype=int)
                b_uvw = np.array([0, 1, 0], dtype=int)
            else:
                # h00 solution
                m = 1
                a_uvw = np.array([0, 1, 0], dtype=int)
                b_uvw = np.array([0, 0, 1], dtype=int)
    elif hkl[1] != 0:
        if hkl[2] != 0:
            # 0kl solution
            m = hkl[1] * hkl[2]
            a_uvw = np.array([0, -m/hkl[1], m/hkl[2]], dtype=int)
            b_uvw = np.array([1, 0, 0], dtype=int)
        else:    
            # 0k0 solution
            m = 1
            a_uvw = np.array([0, 0, 1], dtype=int)
            b_uvw = np.array([1, 0, 0], dtype=int)
    elif hkl[2] != 0:
        # 00l solution
        m = 1
        a_uvw = np.array([1, 0, 0], dtype=int)
        b_uvw = np.array([0, 1, 0], dtype=int)
    else:
        raise ValueError('hkl cannot be all zeros')
    
    for i in range(abs(m), 1, -1):
        test = a_uvw / i
        if np.allclose(test, np.array(test, dtype=int)):
            a_uvw = np.array(test, dtype=int)
        test = b_uvw / i
        if np.allclose(test, np.array(test, dtype=int)):
            b_uvw = np.array(test, dtype=int)
    
    if n is None:
        n = np.max([np.abs(a_uvw), np.abs(b_uvw)])
    
    plane_normal = np.cross(np.inner(a_uvw, box.vects.T), np.inner(b_uvw, box.vects.T))
    plane_normal = plane_normal / np.linalg.norm(plane_normal)
    
    def gen_vector(n):
        for i in range(-n, n+1):
            for j in range(-n, n+1):
                for k in range(-n, n+1):
                    yield np.array([i, j, k])
    
    bestcosine = -1
    for uvw in gen_vector(n=n):
        mag = np.linalg.norm(uvw)
        if mag != 0:
            cosine = np.dot(plane_normal, uvw / mag)
            if np.isclose(cosine, 1, atol=1e-10):
                c_uvw = uvw
                break
            elif cosine > bestcosine:
                bestcosine = cosine
                c_uvw = uvw
    
    for i in range(n, 1, -1):
        test = c_uvw / i
        if np.allclose(test, np.array(test, dtype=int)):
            c_uvw = np.array(test, dtype=int)    
    
    return np.array([a_uvw, b_uvw, c_uvw])

In [18]:
def gen_free_surface_model(hkl, uvws, family, atomshift='0.0 0.0 0.1'):
    """
    Creates free-surface reference data model files.
    
    Parameters
    ----------
    hkl : array-like object
        The three Miller (hkl) indices of the free surface
    uvws : array-like object
        The 3x3 Miller [uvw] indices corresponding to the three system box vectors
    family : str
        The crystal prototype family associated with the free surface
    atomshift : str, optional
        Gives the relative positional shift to apply to all atoms. Default value is
        '0.0, 0.0, 0.1' to ensure an atomic plane is not placed exactly on the cut
        plane.
    """
    
    model_id = family + '--' + '%i%i%i' % tuple(hkl)
    model = DM()
    model['free-surface'] = DM()
    model['free-surface']['key'] = str(uuid.uuid4())
    model['free-surface']['id'] = model_id
    model['free-surface']['system-family'] = family
    model['free-surface']['calculation-parameter'] = DM()
    model['free-surface']['calculation-parameter']['a_uvw'] = '%2i %2i %2i' % tuple(uvws[0])
    model['free-surface']['calculation-parameter']['b_uvw'] = '%2i %2i %2i' % tuple(uvws[1])
    model['free-surface']['calculation-parameter']['c_uvw'] = '%2i %2i %2i' % tuple(uvws[2])
    model['free-surface']['calculation-parameter']['cutboxvector'] = 'c'
    model['free-surface']['calculation-parameter']['atomshift'] = atomshift
    
    with open(model_id + '.json', 'w') as f:
        model.json(fp=f, indent=4)

## Cubic generator

In [24]:
family = 'A4--C--dc'
hkls = [
    [1,0,0],
    [1,1,0],
    [1,1,1],
    [2,1,0],
    [2,1,1],
    [2,2,1],
    [3,1,0],
    [3,1,1],
    [3,2,0],
    [3,2,1],
    [3,2,2],
    [3,3,1],   
    [3,3,2],
       ]

In [25]:
for hkl in hkls:
    gen_free_surface_model(hkl, surface_basis(hkl), family)

## Hexagonal generator